# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd

conn = sqlite3.Connection('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [2]:
# Your code here
pd.read_sql("""
SELECT firstName, lastName
FROM employees
JOIN offices
USING (officeCode)
WHERE city = 'Boston'
""", conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [3]:
# Your code here
pd.read_sql("""
SELECT officeCode, city, COUNT(*) AS num_employees
FROM employees
JOIN offices
USING (officeCode)
GROUP BY officeCode
HAVING num_employees = 0
""", conn)

,officeCode,city,num_employees


## Write 3 questions of your own and answer them

In [4]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
pd.read_sql("""
SELECT officeCode, COUNT(*) AS num_customers
FROM customers
JOIN employees
ON customers.salesRepEmployeeNumber = employees.employeeNumber
JOIN offices
USING (officeCode)
GROUP BY officeCode
""", conn)

,officeCode,num_customers
0,1,12
1,2,12
2,3,15
3,4,29
4,5,5
5,6,10
6,7,17


In [5]:
"""
Question 1
"""
'''
How many employees are there per office?
'''
# Your code here
pd.read_sql("""
SELECT officeCode, COUNT(*) AS num_employees
FROM employees
JOIN offices
USING (officeCode)
GROUP BY officeCode
""", conn)


,officeCode,num_employees
0,1,6
1,2,2
2,3,2
3,4,5
4,5,2
5,6,4
6,7,2


In [6]:
"""
Question 2
"""

'''
which products have been ordered the most?
'''
# Your code here
pd.read_sql("""
SELECT productName, COUNT(*) AS num_ordered
FROM orderdetails
JOIN products
USING (productCode)
GROUP BY productCode
ORDER BY num_ordered DESC
""", conn)

,productName,num_ordered
0,1992 Ferrari 360 Spider red,53
1,Boeing X-32A JSF,28
2,American Airlines: MD-11S,28
3,F/A 18 Hornet 1/72,28
4,ATA: B757-300,28
...,...,...
104,1911 Ford Town Car,25
105,1917 Grand Touring Sedan,25
106,1965 Aston Martin DB5,25
107,1952 Citroen-15CV,24


In [7]:
"""
Question 3
"""
'''
which products have the highest payment?
'''
# Your code here
pd.read_sql("""
SELECT productName, SUM(amount) AS total_payment
FROM products
JOIN orderdetails
USING (productCode)
JOIN orders
USING (orderNumber)
JOIN payments
USING (customerNumber)
GROUP BY productCode
ORDER BY total_payment DESC
""", conn)

,productName,total_payment
0,1992 Ferrari 360 Spider red,10842745.04
1,1958 Chevy Corvette Limited Edition,8428983.28
2,1970 Dodge Coronet,7867836.37
3,1954 Greyhound Scenicruiser,7852167.03
4,Diamond T620 Semi-Skirted Tanker,7241068.96
...,...,...
104,1997 BMW F650 ST,3230427.96
105,ATA: B757-300,3202828.79
106,2002 Suzuki XREO,3166150.70
107,1972 Alfa Romeo GTA,2768327.28


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [8]:
# Your code here
pd.read_sql("""
SELECT firstName, lastName, productName
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
JOIN products
USING (productCode)
""", conn)

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [9]:
# Your code here
pd.read_sql("""
SELECT firstName, lastName, SUM(quantityOrdered) AS total_products_sold
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
ORDER BY lastName
""", conn)

,firstName,lastName,total_products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [10]:
# Your code here
pd.read_sql("""
SELECT firstName, lastName, COUNT(DISTINCT productCode) AS total_products_sold
FROM employees
JOIN customers
ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
USING (customerNumber)
JOIN orderdetails
USING (orderNumber)
GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
HAVING total_products_sold > 200
ORDER BY lastName
""", conn)

,firstName,lastName,total_products_sold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!